In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import squidpy as sq

ModuleNotFoundError: No module named 'squidpy'

In [2]:
import matplotlib.pyplot as plt
import json
import os
import cv2
import time
import numpy as np
from csbdeep.utils import Path, normalize
from segmentation import MELC_Segmentation
import pandas as pd
import anndata as ad

2023-07-17 11:24:19.491866: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-17 11:24:19.492939: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-17 11:24:19.517140: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-17 11:24:19.517759: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-17 11:24:19.926249: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [3]:
data = "ALS"

f = open('/data_slow/je30bery/spatial_proteomics/config.json')
config = json.load(f)
data_path = "/data_slow/je30bery/data/ALS"  #config[data]
seg = MELC_Segmentation(data_path, membrane_marker="cd45") 
# membrane_marker: str/None 
# radius: multiple of cell radius

In [4]:
seg.fields_of_view

['ALS01 - 21297',
 'ALS01 - 21298',
 'ALS01 - 21299',
 'ALS01 - 21300',
 'ALS02 - 21301',
 'ALS02 - 21302',
 'ALS02 - 21303',
 'ALS02 - 21304',
 'ALS03 - 21305',
 'ALS03 - 21306',
 'ALS03 - 21307',
 'ALS03 - 21308',
 'ALS04 - 21370',
 'ALS04 - 21371',
 'ALS04 - 21372',
 'ALS04 - 21373',
 'Healthy01 - 21315',
 'Healthy01 - 21316',
 'Healthy01 - 21317',
 'Healthy01 - 21318',
 'Healthy02 - 21319',
 'Healthy02 - 21320',
 'Healthy02 - 21321',
 'Healthy02 - 21322',
 'Healthy03 - 21323',
 'Healthy03 - 21324',
 'Healthy03 - 21325',
 'Healthy03 - 21326']

In [9]:
def get_expression(adaptive, where_dict):
    expression = np.zeros_like(adaptive)
    expression_dict = dict()
    for n in where_dict:
        if n == 0:
            continue
        
        segment = where_dict[n]
        exp = np.sum(adaptive[segment[0], segment[1]])/len(segment[0])
        expression[segment[0], segment[1]] = exp
        #expression_dict[n] = exp / 255
    return expression

In [5]:
for fov in seg.fields_of_view:
    seg.field_of_view = fov
    nuclei_labels, combined_membranes, nucleus_label_where, membrane_label_where = seg.run()
    if calculate_expression:
        markers = {m.split("_")[1]: os.path.join(seg.get_fov_dir(), m) for m in sorted(os.listdir(seg.get_fov_dir())) if m.endswith(".tif") and "phase" not in m}
        del markers['Propidium iodide']

        for m in markers:
            m_img = cv2.imread(markers[m], cv2.IMREAD_GRAYSCALE) 
            tile_std = np.std(m_img)
            adaptive = cv2.adaptiveThreshold(m_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 201, -tile_std)  
            expression = get_expression(adaptive, where_dict_nuc)
    break

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


2023-07-17 11:24:21.136573: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_SYSTEM_DRIVER_MISMATCH: system has unsupported display driver / cuda driver combination
2023-07-17 11:24:21.136596: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: bionets-scott.aibe.uni-erlangen.de
2023-07-17 11:24:21.136599: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: bionets-scott.aibe.uni-erlangen.de
2023-07-17 11:24:21.136665: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.54.3
2023-07-17 11:24:21.136677: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 530.41.3
2023-07-17 11:24:21.136680: E tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:312] kernel version 530.41.3 does not match DSO version 535.54.3 -- cannot find work

1/1 [==============================] - 1s 632ms/step


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4368/4368 [00:44<00:00, 99.07it/s]


In [8]:
nuclei_labels.shape

(2018, 2018)